In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [2]:
import pandas as pd

In [3]:
from pandas import DataFrame

In [4]:
from numpy import dot
from numpy.linalg import norm

In [5]:
import spacy

In [6]:
import re
import string


In [7]:
nlp = spacy.load('en_core_web_lg')

In [8]:
def getHarmonicScore (accuracy, automationRate, beta):
    
    if(accuracy == 0 and automationRate == 0):
        return 0
    
    score = (1 + beta ** 2) * (accuracy * automationRate) / ((accuracy * (beta ** 2)) + automationRate)
    #s1 = (1 + beta**2)
    #s2 = (accuracy * automationRate)
    #s3 = (accuracy*(beta**2)) + automationRate
    #print(s1)
    #print(s2)
    #print(s3)
    
    return score

In [9]:
def getNumberOfOracled(a):
    counter = 0
    
    for x in a:
        if(x[4] != "predicted"):
            counter = counter + 1
            
    return counter

In [10]:
def getNumberOfPredicted(a):
    counter = 0
    
    for x in a:
        if(x[4] == "predicted"):
            counter = counter + 1
            
    return counter

In [11]:
def cosineSimilarity(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

In [12]:
def getPercentageOfPredicted(a):
    counter = 0
    
    global unclassifiedSamples4GridSearch, classifiedSamples4GridSearch
    
    for x in a:
        if(x[4] == "predicted"):
            counter = counter + 1
            
    return counter / (len(unclassifiedSamples4GridSearch) + len(classifiedSamples4GridSearch))

In [13]:
def getPercentageCorrectPredictions(a):
    
    global unclassifiedSamples4GridSearch, classifiedSamples4GridSearch
    
    counter = 0
    for x in a:
        if(x[1] == x[3]):
            counter = counter + 1

    counter = counter + len(unclassifiedSamples4GridSearch)
            
    return counter / (len(unclassifiedSamples4GridSearch) + len(classifiedSamples4GridSearch))
            
    

In [14]:
def getLocalAutomationRate(a):
    counter = 0
    
    global unclassifiedSamples4GridSearch, classifiedSamples4GridSearch
    
    for x in a:
        if(x[4] == "predicted"):
            counter = counter + 1
            
    return counter / (len(a))

In [15]:
def getLocalAccuracy(a):
    
    counter = 0
    for x in a:
        if(x[1] == x[3]):
            counter = counter + 1

    
            
    return counter / (len(classifiedSamples4GridSearch))

In [16]:
def preprocessing(sentence):
    
    doc = nlp(sentence)
    inputSent = ""

    #removing stop words
    for token in doc:
        
        if(token.is_stop == True):
            continue
        else:
            inputSent = inputSent + " " + token.lemma_
        
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
    
    #lower case of the sentence
    inputSent = inputSent.lower()
    #remove numbers
    inputSent = re.sub('\d+', '', inputSent)
    #remover punctuation
    inputSent = inputSent.translate(str.maketrans('', '', string.punctuation))
    #remove multiple whitespaces
    inputSent = ' '.join(inputSent.split())
    
    
    
    #print(inputSent)
    return str(inputSent)

In [17]:
df = pd.read_csv('IBM_shuffled.csv', delimiter=",")


In [151]:
df = pd.read_csv('HR_IT_FM_shuffled.csv', delimiter = ";")

In [155]:
df = pd.read_csv('HR_IT_FM_shuffled_small1.csv', delimiter = ";")

In [156]:
df.head(5)

,Text,Label
0,"Hello, the airconditioning system in my office...",Facility Management
1,"Hey, I know that some departments were moved d...",Facility Management
2,Good afternoon. Could you please update the on...,Facility Management
3,"Hi, I have a following question. I am unable t...",IT
4,when my colleagues make any changes to the sha...,IT


In [157]:
uniqueLabels = df.Label.unique()

foundLabels = []
classifiedSamplesInit = []

tfIdfList = []
allSamplesInit = []
allSamplesWithLabelInit = []

allSamplesMed = []
allSamplesWithLabelMed = []

allSamplesThird = []
allSamplesWithLabelThird = []

allSamplesOracled = []

In [158]:
#Save information from df in variables. Hereby, we compute the tf-idf values and eliminate stop words, numbers and more.

for ind in df.index:
    preprocessedSentence = preprocessing(df['Text'][ind])
    #preprocessedSentence = (df['Text'][ind])
    allSamplesWithLabelInit.append([preprocessedSentence, df['Label'][ind]])
    allSamplesInit.append(preprocessedSentence)

#Get TF-IDF Values
analyze = vectorizer.build_analyzer()
X = vectorizer.fit_transform(allSamplesInit)
(vectorizer.get_feature_names())
tfIdfList = X.toarray()

In [159]:
#Set up a list for further work with tuples of the format (text, label, tf-idf-values)
counter = 0
for sample in allSamplesWithLabelInit:
    text = sample[0]
    lable = sample[1]
    tfIdfVal = tfIdfList[counter]
    #embedding = nlp(text).vector 
    #embedding = getTfIdfSentenceEmbedding(text, counter)
    #embedding = getFasttextSentenceEmbedding(text, counter, False)
    #embedding = model.encode(text)
    allSamplesWithLabelMed.append([text, lable, tfIdfVal, 0])
    
    counter = counter + 1 
    
#The oracle assigns the label for one sample per category    

for x in allSamplesWithLabelMed:

    wholeEntry = [x[0], x[1], x[2], x[1], "oracle", x[3]]
    
    if(not(x[1] in foundLabels)):
        foundLabels.append(x[1])
        classifiedSamplesInit.append(wholeEntry)
    else: 
        allSamplesWithLabelThird.append(x)
    
    allSamplesOracled.append(wholeEntry)
 

sentenceDict = {}
similarityDict = {}

for label in uniqueLabels:
    sentenceDict.update({label: ""})
    
for label in uniqueLabels:
    similarityDict.update({label: 0})
    
#print(similarityDict, '\n \n')

In [160]:
#len(allSamplesWithLabelThird)

In [161]:
#len(classifiedSamplesInit)

In [162]:
'''
STRUCTURE OF allSamplesWithLabelThird: ticketText, ticketLabel, TF-IDF

STRUCTURE OF classifiedSamplesInit: ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, embedding
'''

'\nSTRUCTURE OF allSamplesWithLabelThird: ticketText, ticketLabel, TF-IDF\n\nSTRUCTURE OF classifiedSamplesInit: ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, embedding\n'

In [163]:
def addOracledSamples(listOfSamples, samplesPerCategory):
    
    startDict = {}
    
    for label in uniqueLabels:
        startDict.update({label: 0})
        
    classifiedSamples1 = []
        
    limit = samplesPerCategory * len(uniqueLabels)
    
    unclassifiedList = listOfSamples[:]
    
    unclassifiedListFinal = []
    
    #print(limit, len(unclassifiedList))
    #print(startDict)
    
    for x in unclassifiedList:
        #ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, embedding
        wholeEntry = [x[0], x[1], x[2], x[1], "oracle", x[5]]
        
        amountOfSamplesPerLabel = startDict.get(x[1])
        
        if(amountOfSamplesPerLabel < samplesPerCategory):
            #print(x[1], amountOfSamplesPerLabel)
            classifiedSamples1.append(wholeEntry)
            amountOfSamplesPerLabel = amountOfSamplesPerLabel + 1
            startDict.update({x[1]: amountOfSamplesPerLabel})
            
            
            
            unclassifiedList.remove(x)
            
            
        if(limit == len(classifiedSamples1)):
            break
            
    for x in unclassifiedList:
        text = x[0]
        lable = x[1]
        tfIdfVal = x[2]
        
        unclassifiedListFinal.append([text, lable, tfIdfVal, x[5]])
    
            
            
    print(len(classifiedSamples1), len(unclassifiedListFinal))
    
    #helper = [i[1] for i in classifiedSamples1]
    
    #print(helper)
    
    #print(startDict) 
    
    return classifiedSamples1, unclassifiedListFinal
            
    

In [164]:
allUnclassifiedSamples = allSamplesWithLabelThird[:]
allClassifiedSamples = classifiedSamplesInit[:]

In [165]:
def resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2):
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch
    
    
    consideredSamples = allSamplesOracled[startIndex1:endIndex1] + allSamplesOracled[startIndex2:endIndex2]
    
    print(len(consideredSamples))
    
    classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(consideredSamples, 1)

In [166]:
classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)

3 197


In [167]:
resetClassifiedSamples(0, 300, 0, 0)

300
3 297


In [168]:
#Hereby, if we have 4 categories, we save the biggest similarity per each category with the new sample 
#If the similarites are more or less the same (DEFINE Threshold), then 
def findAndClassifyEqualSimilaritySamples(similarityHP, gapHP):
    
    maxSimilarityPerCategories = []
    
    for label in uniqueLabels:
        maxSimilarityPerCategories.append((label, 0))
    #print(maxSimilarityPerCategories)
    
    
    for unclassifiedSample in unclassifiedSamples4GridSearch:
    
        mostSimilarSample = 0
        maxCosSim = 0
        maxCosSimLabel = ""
        
        for label in uniqueLabels:
            similarityDict.update({label: 0})
    
        for classifiedSample in classifiedSamples4GridSearch:
            cosSim = cosineSimilarity(unclassifiedSample[2], classifiedSample[2])
            #print(len(unclassifiedSample), len(classifiedSample))
            #cosSim = computeSBertSimilarity((unclassifiedSample[3]), (classifiedSample[5]))
            classifiedLabel = classifiedSample[1]
            
            if(maxCosSim < cosSim):
                maxCosSim = cosSim
                maxCosSimLabel = classifiedLabel
            
            if(cosSim > similarityDict[classifiedLabel]):
                similarityDict [classifiedLabel] = cosSim
                sentenceDict [classifiedLabel] = unclassifiedSample[0]
        
        gapPassed = False
        
        for i in similarityDict:
            if(i != maxCosSimLabel):
                gap = maxCosSim - similarityDict[i]
                if(gap < gapHP):
                    gapPassed = True
                    break
        
        if(maxCosSim > similarityHP and gapPassed):
            # ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted
            entry = [unclassifiedSample[0], unclassifiedSample[1], unclassifiedSample[2], unclassifiedSample[1], "oracle", unclassifiedSample[3]]
            #print("entry ", entry)
        
            try:
                unclassifiedSamples4GridSearch.remove(unclassifiedSample)
                classifiedSamples4GridSearch.append(entry)
            except:
                print("entry ", entry)
        
        elif(maxCosSim > similarityHP):
            #STRUCTURE OF classifiedSamplesInit: 
            #ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, wordVector
            entry = [unclassifiedSample[0], maxCosSimLabel, unclassifiedSample[2], unclassifiedSample[1], "predicted", unclassifiedSample[3]]
            #print("entry ", entry)
            
            try:
                unclassifiedSamples4GridSearch.remove(unclassifiedSample)
                classifiedSamples4GridSearch.append(entry)
            except:
                print("entry ", entry)
            
            
            
 
        #print(similarityDict, unclassifiedSample[1])
        #print()
    #print("FINISHED ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
    #print()
    return len(unclassifiedSamples4GridSearch)

In [169]:
#findAndClassifyEqualSimilaritySamples()

In [170]:
def gridSearchAnd3FoldValidation (minSimilarity, maxSimilarity, similarityStep, minGap, maxGap, gapStep, startIndex1, endIndex1, startIndex2, endIndex2):
    
    similarityGrid = []
    gapGrid = []
    
    allResults = []
    dfColumnName = ["Accuracy", "Automation Rate", "Harmonic Score", "Local Accuracy", "Local Automation Rate", "Local Harmonic Score", "Number Of Oracled", "Number Of Predicted", "Minimal Similarity", "Minimal Gap"]
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch
    
    
    similarity = minSimilarity
    
    while(similarity <= maxSimilarity):
        similarityGrid.append(similarity)
        similarity = similarity + similarityStep
        
    g = minGap

    while(g <= maxGap):
        gapGrid.append(g)
        g = g + gapStep
        
    print("Gap Grid: ", gapGrid)
    print("Similarity Grid: ", similarityGrid)
    print("\n")
    #print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore," 
    #      "numberOfOracled, numberOfPredicted, sim, gap, currentSim, currentGap")
    print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap")
    
    
    maxHarmonicScore = 0
    bestGap = 0
    bestSim = 0
    bestAccuracy = 0
    bestAutomationRate = 0
        
    for sim in similarityGrid:
    
        
        for gap in gapGrid:
            
            print("NOW NEW COMBINATION: ", sim, gap)
            
            numberOfPredicted = -1
            numberOfOracled = -1

            localAccuracy = -1
            globalAccuracy = -1
            localAutomationRate = -1
            globalAutomationRate = -1

            localHarmonicScore = -1
            globalHarmonicScore = -1

            unlabelledSamples = 1000
            
            currentSim = sim
            currentGap = gap
            
            simCounter = 0
            gapCounter = 0
            counter = 0
            
            resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
            
            while (unlabelledSamples != 0 and counter < 55):
                
                
                #print("sim, gap ", currentSim, currentGap)
                unlabelledSamples = findAndClassifyEqualSimilaritySamples(currentSim, currentGap)      
            
                numberOfNewPredicted = getNumberOfPredicted(classifiedSamples4GridSearch)
                numberOfNewOracled = getNumberOfOracled(classifiedSamples4GridSearch)
                
                if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted):
                    currentSim = currentSim - 0.01
                    simCounter = simCounter + 1
                    
                '''
                if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted and simCounter > 12):
                    #currentGap = currentGap - 0.005
                    gapCounter = gapCounter + 1
                '''
                
                numberOfOracled = numberOfNewOracled
                numberOfPredicted = numberOfNewPredicted

                if(unlabelledSamples == 0):
                    counter = counter + 10000
                    
                print("Automation Rate ",getPercentageOfPredicted(classifiedSamples4GridSearch))
                print("Accuracy ",getPercentageCorrectPredictions(classifiedSamples4GridSearch))
                print("Number of Predicted ", getNumberOfPredicted(classifiedSamples4GridSearch))
                print("Number of Oracled ",getNumberOfOracled(classifiedSamples4GridSearch))
                
                #accuracy, automationRate, numberOfOracled, numberOfPredicted, currentSim, currentGap
                counter = counter + 1 
            
            #harmonicScore = getHarmonicScore(localAccuracy, localAutomationRate, 0.5)
            
            localAccuracy = getLocalAccuracy(classifiedSamples4GridSearch)
            localAutomationRate = getLocalAutomationRate(classifiedSamples4GridSearch)
            localHarmonicScore = getHarmonicScore(localAccuracy, localAutomationRate, 0.5)
            
            globalAutomationRate = getPercentageOfPredicted(classifiedSamples4GridSearch)
            globalAccuracy  = getPercentageCorrectPredictions(classifiedSamples4GridSearch)
            globalHarmonicScore = getHarmonicScore(globalAccuracy, globalAutomationRate, 0.5)
            
            
            if(globalHarmonicScore > maxHarmonicScore):
                maxHarmonicScore = localHarmonicScore
                bestAccuracy = localAccuracy
                bestAutomationRate = localAutomationRate
                bestGap = gap
                bestSim = sim
            
                
            #print("")
            #print("before reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
            
            
            
            #classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)
            #resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
            #classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)
            #print("after reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
            #print("")
            
            print(globalAccuracy, globalAutomationRate, globalHarmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap)
            
            tempResults = [globalAccuracy, globalAutomationRate, globalHarmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap]
            allResults.append(tempResults)
            
    print("Best: ", maxHarmonicScore, bestAccuracy, bestAutomationRate, bestGap, bestSim)
    return allResults, dfColumnName
            

In [171]:
def validation (similarityGapGrid, startIndex1, endIndex1, startIndex2, endIndex2):
    
    similarityGrid = []
    gapGrid = []
    
    allResults = []
    dfColumnName = ["Accuracy", "Automation Rate", "Harmonic Score", "Local Accuracy", "Local Automation Rate", "Local Harmonic Score", "Number Of Oracled", "Number Of Predicted", "Minimal Similarity", "Minimal Gap"]
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch

        
    print("Gap Grid: ", gapGrid)
    print("Similarity Grid: ", similarityGrid)
    print("\n")
    #print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore," 
    #      "numberOfOracled, numberOfPredicted, sim, gap, currentSim, currentGap")
    print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap")
    
    
    maxHarmonicScore = 0
    bestGap = 0
    bestSim = 0
    bestAccuracy = 0
    bestAutomationRate = 0
    
    
    
        
    for simGap in similarityGapGrid:
        
        numberOfPredicted = -1
        numberOfOracled = -1
        accuracy = -1
        automationRate = -1
        unlabelledSamples = 1000

        currentSim = simGap[0]
        currentGap = simGap[1]


        simCounter = 0
        gapCounter = 0
        counter = 0

        while (unlabelledSamples != 0 and counter < 100):

            #print("during process ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))

            unlabelledSamples = findAndClassifyEqualSimilaritySamples(currentSim, currentGap)


            numberOfNewPredicted = getNumberOfPredicted(classifiedSamples4GridSearch)
            numberOfNewOracled = getNumberOfOracled(classifiedSamples4GridSearch)
            automationRate = getPercentageOfPredicted(classifiedSamples4GridSearch)
            accuracy = getPercentageCorrectPredictions(classifiedSamples4GridSearch)

            if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted):
                currentSim = currentSim - 0.01
                simCounter = simCounter + 1
                #currentGap = currentGap - 1

            if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted and simCounter > 12):
                currentGap = currentGap - 0.005
                gapCounter = gapCounter + 1

            numberOfOracled = numberOfNewOracled
            numberOfPredicted = numberOfNewPredicted

            #print("unlab samples ", unlabelledSamples)

            if(unlabelledSamples == 0):
                break

            #accuracy, automationRate, numberOfOracled, numberOfPredicted, currentSim, currentGap
            counter = counter + 1 

        harmonicScore = getHarmonicScore(accuracy, automationRate, 0.5)

        localAccuracy = getLocalAccuracy(classifiedSamples4GridSearch)
        localAutomationRate = getLocalAutomationRate(classifiedSamples4GridSearch)
        localHarmonicScore = getHarmonicScore(localAccuracy, localAutomationRate, 0.5)

        if(harmonicScore > maxHarmonicScore):
            maxHarmonicScore = harmonicScore
            bestAccuracy = accuracy
            bestAutomationRate = automationRate
            bestSim = simGap[0]
            bestGap = simGap[1]


        #print("")
        print("before reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))

        resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
        #classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)
        print("after reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
        #print("")

        print(accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, simGap[0], simGap[1])

        tempResults = [accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, simGap[0], simGap[1]]
        allResults.append(tempResults)
            
    print("Best: ", maxHarmonicScore, bestAccuracy, bestAutomationRate, bestGap, bestSim)
    return allResults, dfColumnName
            

In [36]:
#startIndex1, endIndex1, startIndex2, endIndex2 = 0, 200, 0, 0
#gridSearchAnd3FoldValidation(minSimilarity = 0.20, maxSimilarity = 0.26, similarityStep = 0.01 , minGap = 0.03, maxGap = 0.03, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

#import winsound
#frequency = 1000  # Set Frequency To 2500 Hertz
#duration = 3000  # Set Duration To 1000 ms == 1 second
#winsound.Beep(frequency, duration)

In [44]:
startIndex1, endIndex1, startIndex2, endIndex2 = 0,200,0,0
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults1, tfIdfColumns1 = gridSearchAnd3FoldValidation(minSimilarity = 0.05, maxSimilarity = 0.91, similarityStep = 0.05 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 100,300,0,0
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults2, tfIdfColumns2 = gridSearchAnd3FoldValidation(minSimilarity = 0.05, maxSimilarity = 0.91, similarityStep = 0.05 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 0,100,200,300
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults3, tfIdfColumns3 = gridSearchAnd3FoldValidation(minSimilarity = 0.05, maxSimilarity = 0.91, similarityStep = 0.05 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

#resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
#tfIdfResults, tfIdfColumns = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.3, similarityStep = 0.06 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 3500  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

200
15 185
Gap Grid:  [0.01, 0.015, 0.02, 0.025, 0.030000000000000002, 0.035, 0.04, 0.045, 0.049999999999999996]
Similarity Grid:  [0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.3, 0.35, 0.39999999999999997, 0.44999999999999996, 0.49999999999999994, 0.5499999999999999, 0.6, 0.65, 0.7000000000000001, 0.7500000000000001, 0.8000000000000002, 0.8500000000000002, 0.9000000000000002]


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
NOW NEW COMBINATION:  0.05 0.01
200
15 185
entry  ['hey know department move facility base change company tell hr team place sign contract time extend', 'IT', array([0., 0., 0., ..., 0., 0., 0.]), 'Facility Management', 'predicted', 0]
entry  ['open office cabinet think lose key help inside important document thank', 'HR', array([0., 0., 0., ..., 0., 0., 0.]), 'Facility Management', 'predicted', 0]
entry  ['frosty hello walldorf colleague recognize heating system work sha

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [66]:
df1 = DataFrame (tfIdfResults1, columns= tfIdfColumns1)
df2 = DataFrame (tfIdfResults2, columns= tfIdfColumns2)
df3 = DataFrame (tfIdfResults3, columns= tfIdfColumns3)
df1

,Accuracy,Automation Rate,Harmonic Score,Local Accuracy,Local Automation Rate,Local Harmonic Score,Number Of Oracled,Number Of Predicted,Minimal Similarity,Minimal Gap
0,0.690,0.770,0.704642,0.690,0.770000,0.704642,46,154,0.05,0.010
1,0.705,0.735,0.710802,0.705,0.735000,0.710802,53,147,0.05,0.015
2,0.780,0.695,0.761376,0.780,0.695000,0.761376,61,139,0.05,0.020
3,0.805,0.660,0.771118,0.805,0.660000,0.771118,68,132,0.05,0.025
4,0.830,0.620,0.777341,0.830,0.620000,0.777341,76,124,0.05,0.030
...,...,...,...,...,...,...,...,...,...,...
157,1.000,0.005,0.024510,1.000,0.047619,0.200000,20,1,0.90,0.030
158,1.000,0.005,0.024510,1.000,0.047619,0.200000,20,1,0.90,0.035
159,1.000,0.005,0.024510,1.000,0.047619,0.200000,20,1,0.90,0.040
160,1.000,0.005,0.024510,1.000,0.047619,0.200000,20,1,0.90,0.045


In [68]:
with pd.ExcelWriter("TF-IDF-5PerCat-CFV.xlsx") as writer:
    df1.to_excel(writer, sheet_name='1', index=False)
    df2.to_excel(writer, sheet_name='2', index=False)
    df3.to_excel(writer, sheet_name='3', index=False)

In [ ]:
import os
os.system("shutdown.exe /h")


In [34]:
#1 per Cat

startIndex1, endIndex1, startIndex2, endIndex2 = 200,300,0,0
simGapGrid1 = [[0.1, 0.04], [0.1, 0.035], [0.3, 0.05], [0.35, 0.05]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults1, tfIdfColumns1 = validation(simGapGrid1, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 0,100,0,0
simGapGrid2 = [[0.1, 0.025], [0.1, 0.03], [0.15, 0.05], [0.15, 0.04], [0.1, 0.035]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults2, tfIdfColumns2 = validation(simGapGrid2, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 100,200,0,0
simGapGrid3 = [[0.05, 0.045], [0.05, 0.04], [0.05, 0.035], [0.05, 0.05]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults3, tfIdfColumns3 = validation(simGapGrid3, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)
#resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
#tfIdfResults, tfIdfColumns = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.3, similarityStep = 0.06 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 5000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

100
4 96
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
before reset  0 100
100
4 96
after reset  96 4
0.84 0.54 0.7559999999999999 0.84 0.54 0.7559999999999999 46 54 0.1 0.04
before reset  0 100
100
4 96
after reset  96 4
0.77 0.65 0.7425816023738874 0.77 0.65 0.7425816023738874 35 65 0.1 0.035
before reset  0 100
100
4 96
after reset  96 4
0.62 0.65 0.6257763975155279 0.62 0.65 0.6257763975155279 35 65 0.3 0.05
before reset  0 100
100
4 96
after reset  96 4
0.43 0.89 0.4795739348370927 0.43 0.89 0.4795739348370927 11 89 0.35 0.05
Best:  0.7559999999999999 0.84 0.54 0.04 0.1
100
4 96
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
before reset  0 100
100
4 96
after reset  96 4
0.68 0.71 0.6857954545454545 0.68 0.71 0.685795454

In [37]:
#2 per Cat

startIndex1, endIndex1, startIndex2, endIndex2 = 200,300,0,0
simGapGrid1 = [[0.05, 0.03], [0.05, 0.035], [0.5, 0.04], [0.1, 0.4]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults1, tfIdfColumns1 = validation(simGapGrid1, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 0,100,0,0
simGapGrid2 = [[0.05, 0.03], [0.05, 0.035], [0.05, 0.04], [0.05, 0.045]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults2, tfIdfColumns2 = validation(simGapGrid2, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 100,200,0,0
simGapGrid3 = [[0.05, 0.03], [0.05, 0.035], [0.05, 0.04], [0.05, 0.045]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults3, tfIdfColumns3 = validation(simGapGrid3, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)
#resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
#tfIdfResults, tfIdfColumns = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.3, similarityStep = 0.06 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 5000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

100
8 92
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
before reset  0 100
100
8 92
after reset  92 8
0.78 0.6 0.7358490566037736 0.78 0.6 0.7358490566037736 40 60 0.05 0.03
before reset  0 100
100
8 92
after reset  92 8
0.82 0.58 0.7573248407643312 0.82 0.58 0.7573248407643312 42 58 0.05 0.035
before reset  0 100
100
8 92
after reset  92 8
0.34 0.92 0.3890547263681592 0.34 0.92 0.3890547263681592 8 92 0.5 0.04
before reset  0 100
100
8 92
after reset  92 8
1.0 0.0 0.0 1.0 0.0 0.0 100 0 0.1 0.4
Best:  0.7573248407643312 0.82 0.58 0.035 0.05
100
8 92
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
before reset  0 100
100
8 92
after reset  92 8
0.85 0.53 0.7584175084175083 0.85 0.53 0.7584175084175083 47 53 0.05 0.03
before rese

In [38]:
d11 = DataFrame (tfIdfResults1, columns= tfIdfColumns1)
d22 = DataFrame (tfIdfResults2, columns= tfIdfColumns2)
d33 = DataFrame (tfIdfResults3, columns= tfIdfColumns3)

In [39]:
with pd.ExcelWriter("TF-IDF-3PerCat-Validation.xlsx") as writer:
    d11.to_excel(writer, sheet_name='1', index=False)
    d22.to_excel(writer, sheet_name='2', index=False)
    d33.to_excel(writer, sheet_name='3', index=False)

In [41]:

startIndex1, endIndex1, startIndex2, endIndex2 = 300,400,0,0
#to be changed
simGapGrid1 = [[0.1, 0.04], [0.1, 0.035], [0.05, 0.05], [0.05, 0.045], [0.05, 0.04]]

#2 per category
simGapGrid1 =[[0.05, 0.03], [0.05, 0.035], [0.05, 0.04], [0.05, 0.045], [0.1, 0.04]]

resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults111, tfIdfColumns111 = validation(simGapGrid1, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)


import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 3500  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

df111 = DataFrame (tfIdfResults111, columns= tfIdfColumns111)

100
8 92
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
before reset  0 100
100
8 92
after reset  92 8
0.8 0.54 0.7297297297297298 0.8 0.54 0.7297297297297298 46 54 0.05 0.03
before reset  0 100
100
8 92
after reset  92 8
0.8 0.52 0.7222222222222223 0.8 0.52 0.7222222222222223 48 52 0.05 0.035
before reset  0 100
100
8 92
after reset  92 8
0.82 0.49 0.7226618705035971 0.82 0.49 0.7226618705035971 51 49 0.05 0.04
before reset  0 100
100
8 92
after reset  92 8
0.83 0.44 0.705019305019305 0.83 0.44 0.705019305019305 56 44 0.05 0.045
before reset  0 100
100
8 92
after reset  92 8
0.83 0.46 0.7149812734082397 0.83 0.46 0.7149812734082397 54 46 0.1 0.04
Best:  0.7297297297297298 0.8 0.54 0.03 0.05


In [42]:
with pd.ExcelWriter("TF-IDF-3perCat-Final.xlsx") as writer:
    df111.to_excel(writer, sheet_name='1', index=False)

In [466]:
getPercentageOfPredicted(allClassifiedSamples)

0.9512195121951219

In [183]:
resetClassifiedSamples()
print("lengths ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))

4 196
lengths  196 4


In [40]:
startIndex1, endIndex1, startIndex2, endIndex2 = 300, 400, 0, 0
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)

100
4 96


In [53]:
similarityHP = 0.05
gapHP = 0.045

#resetClassifiedSamples(0,200,0,0)
findAndClassifyEqualSimilaritySamples(similarityHP, gapHP)

0

In [71]:
print("Automation Rate ",getPercentageOfPredicted(classifiedSamples4GridSearch))
print("Accuracy ",getPercentageCorrectPredictions(classifiedSamples4GridSearch))
print("Number of Predicted ", getNumberOfPredicted(classifiedSamples4GridSearch))
print("Number of Oracled ",getNumberOfOracled(classifiedSamples4GridSearch))

Automation Rate  0.08
Accuracy  0.9766666666666667
Number of Predicted  24
Number of Oracled  49


In [131]:
getPercentageCorrectPredictions(allClassifiedSamples)

0.09

In [468]:
getNumberOfPredicted(allClassifiedSamples)

78

In [469]:
getNumberOfOracled(allClassifiedSamples)

4

In [247]:
# If beta = 1, then accuracy and automationRate have same importance
# If beta < 1, then accuracy is more important than automation rate
# If beta > 1, then automation rate is more important than accuracy
def getHarmonicScore (accuracy, automationRate, beta):
    
    if(accuracy == 0 and automationRate == 0):
        return 0
    
    score = (1 + beta ** 2) * (accuracy * automationRate) / ((accuracy * (beta ** 2)) + automationRate)
    #s1 = (1 + beta**2)
    #s2 = (accuracy * automationRate)
    #s3 = (accuracy*(beta**2)) + automationRate
    #print(s1)
    #print(s2)
    #print(s3)
    
    return score

In [248]:
getHarmonicScore(0.00, 0.00, 0.5)

0

In [30]:
0.9*(0.5**2)

0.225

In [210]:
df

,Unnamed: 0,Text,Label
0,268,"Hello, I have called Citibank for a total of t...",Credit_Card
1,274,I have a Bank of America Visa card. I had a ba...,Credit_Card
2,385,We refinanced our home with Bank of America. L...,Bank_Account
3,317,I have been reviewing my bank statements and h...,Bank_Account
4,101,"""I consolidated my loans several years ago for...",Loans
...,...,...,...
395,32,My loan was sold ( again ) and serviced by Loa...,Mortgage
396,319,I wanted to stop automatic monthly withdrawals...,Bank_Account
397,372,We got a X fee to our checking account but cou...,Bank_Account
398,108,"""I called the Navient Private Loan line on , ...",Loans


In [172]:
classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:], 1)

3 333


In [173]:
def resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2):
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch
    
    
    consideredSamples = allSamplesOracled[startIndex1:endIndex1] + allSamplesOracled[startIndex2:endIndex2]
    
    print(len(consideredSamples))
    
    classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(consideredSamples, 1)

In [174]:
#1 per Cat

startIndex1, endIndex1, startIndex2, endIndex2 = 0, 600,0,0
simGapGrid1 = [[0.05, 0.05], [0.05, 0.045], [0.05, 0.04], [0.05, 0.055], [0.05, 0.06], [0.05, 0.065], [0.05, 0.07]]
#resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults1, tfIdfColumns1 = validation(simGapGrid1, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 3000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
before reset  0 336
336
3 333
after reset  333 3
0.8898809523809523 0.7232142857142857 0.8506729796560639 0.8898809523809523 0.7232142857142857 0.8506729796560639 93 243 0.05 0.05
before reset  0 336
336
3 333
after reset  333 3
0.8839285714285714 0.7410714285714286 0.8511145177328473 0.8839285714285714 0.7410714285714286 0.8511145177328473 87 249 0.05 0.045
before reset  0 336
336
3 333
after reset  333 3
0.8601190476190477 0.7589285714285714 0.8377782931354361 0.8601190476190477 0.7589285714285714 0.8377782931354361 81 255 0.05 0.04
before reset  0 336
336
3 333
after reset  333 3
0.9017857142857143 0.6964285714285714 0.8515652023521273 0.9017857142857143 0.6964285714285714 0.8515652023521273 102 234 0.05 0.055
before reset  0 336
336
3 333
after reset  333 3
0.9166666666666666 0.6636904761904762 0.851736111

In [175]:
d1111 = DataFrame (tfIdfResults1, columns= tfIdfColumns1)


In [176]:
with pd.ExcelWriter("TF-IDF-OwnDataSet-h1-1unbalanced.xlsx") as writer:
    d1111.to_excel(writer, sheet_name='1', index=False)